In [1]:
!pip install --upgrade google-cloud-aiplatform
!pip install --upgrade google-genai
!pip install --upgrade pandas-gbq

In [2]:
from google import genai
from google.genai import types
import base64
from google.cloud import storage
from google.cloud import bigquery
from tenacity import retry, wait_random_exponential
from datetime import datetime
import pandas_gbq


In [3]:
project_id = "cloud-llm-preview2"
location = "us" # Format is "us" or "eu"
region ="us-central1"
mime_type = "application/pdf" # Refer to https://cloud.google.com/document-ai/docs/file-types for supported file types
# bucket_name = "contractanalysis-demo-sg"
gcs_input_uri = "gs://bh-invoice-raw-sgg"
prospectus_bucket = "bh-invoice-raw-sg"
model = "gemini-2.5-pro"
DATASET_ID = "bh_invoice_sg"
INVOICE_TABLE_ID = "invoicedetail"
ADDRESS_TABLE_ID = "invoiceaddress"
SUPPLIERCODING_TABLE_ID = "suppliercoding"
SUPPLIERCODINGDB_TABLE_ID = "suppliercodingdb"


In [4]:
from google.colab import auth

auth.authenticate_user(project_id=project_id)

#**Setup variables**

In [5]:
system_instruction = """You are an expert invoice processing agent at a hedgefund/Asset Manager. Your primary function is to accurately extract key information from invoice documents and return the data in a structured JSON format.
Instructions:
Analyze the entire invoice document provided as input.
Identify and extract the specific data fields listed in the JSON schema below.
Populate the JSON object with the extracted information.
Adhere strictly to the specified data types. Monetary values and quantities should be numbers (integers or floats), not strings with currency symbols. Dates should be in 'YYYY-MM-DD' format.
If a specific field is not present on the invoice, the corresponding value in the JSON output should be null. Do not invent or infer data that isn't explicitly stated.
The output must be a single, valid JSON object:
{
    \"type\": \"OBJECT\",
    \"properties\": {
        \"CustomerName\": {
            \"type\": \"STRING\"
        },
        \"BillingAddress\": {
            \"type\": \"STRING\"
        },
        \"ShippingAddress\": {
            \"type\": \"STRING\"
        },
        \"CustomerAccount_ReferenceNumber\": {
            \"type\": \"STRING\"
        },
        \"VendorName\": {
            \"type\": \"STRING\"
        },
        \"VendorAddress\": {
            \"type\": \"STRING\"
        },
        \"VendorContactInformation\": {
            \"type\": \"STRING\"
        },
        \"VendorVAT_TaxRegistrationNumber\": {
            \"type\": \"STRING\"
        },
        \"TerminationNotificationPeriod\": {
            \"type\": \"STRING\"
        },
        \"InvoiceDate\": {
            \"type\": \"STRING\",
            \"format\": \"date\"
        },
        \"PaymentDueDate\": {
            \"type\": \"STRING\",
            \"format\": \"date\"
        },
        \"PurchaseOrderNumber\": {
            \"type\": \"STRING\"
        },
        \"OrderDate\": {
            \"type\": \"STRING\",
            \"format\": \"date\"
        },
        \"SubTotal_NetAmount\": {
            \"type\": \"OBJECT\",
            \"properties\": {
                \"Value\": {
                    \"type\": \"NUMBER\"
                },
                \"CurrencyCode\": {
                    \"type\": \"STRING\"
                }
            }
        },
        \"TAX_VATAmmount\": {
            \"type\": \"OBJECT\",
            \"properties\": {
                \"Value\": {
                    \"type\": \"NUMBER\"
                },
                \"CurrencyCode\": {
                    \"type\": \"STRING\"
                }
            }
        },
        \"TotalAmmountDue\": {
            \"type\": \"OBJECT\",
            \"properties\": {
                \"Value\": {
                    \"type\": \"NUMBER\"
                },
                \"CurrencyCode\": {
                    \"type\": \"STRING\"
                }
            }
        },
        \"TAX_VATRate\": {
            \"type\": \"NUMBER\"
        },
        \"Discounts_Promotions\": {
            \"type\": \"NUMBER\"
        },
        \"Shipping_DeliveryCharges\": {
            \"type\": \"OBJECT\",
            \"properties\": {
                \"Value\": {
                    \"type\": \"NUMBER\"
                },
                \"CurrencyCode\": {
                    \"type\": \"STRING\"
                }
            }
        },
        \"LineItemDetails\": {
            \"type\": \"ARRAY\",
            \"items\": {
                \"type\": \"OBJECT\",
                \"properties\": {
                    \"Description\": {
                        \"type\": \"STRING\"
                    },
                    \"Quantity\": {
                        \"type\": \"NUMBER\"
                    },
                    \"UnitPrice\": {
                        \"type\": \"OBJECT\",
                        \"properties\": {
                            \"Value\": {
                                \"type\": \"NUMBER\"
                            },
                            \"CurrencyCode\": {
                                \"type\": \"STRING\"
                            }
                        }
                    },
                    \"LineItemTotal\": {
                        \"type\": \"OBJECT\",
                        \"properties\": {
                            \"Value\": {
                                \"type\": \"NUMBER\"
                            },
                            \"CurrencyCode\": {
                                \"type\": \"STRING\"
                            }
                        }
                    },
                    \"Product_SKU Code\": {
                        \"type\": \"STRING\"
                    }
                }
            }
        }
    },
    \"required\": [
        \"CustomerName\",
        \"BillingAddress\",
        \"ShippingAddress\",
        \"CustomerAccount_ReferenceNumber\",
        \"VendorName\",
        \"VendorAddress\",
        \"VendorContactInformation\",
        \"VendorVAT_TaxRegistrationNumber\",
        \"TerminationNotificationPeriod\",
        \"PaymentDueDate\",
        \"PurchaseOrderNumber\",
        \"OrderDate\",
        \"SubTotal_NetAmount\",
        \"TAX_VATAmmount\",
        \"TotalAmmountDue\",
        \"TAX_VATRate\",
        \"Discounts_Promotions\",
        \"Shipping_DeliveryCharges\",
        \"LineItemDetails\"
    ]
}”"""

In [6]:
prompt = types.Part.from_text(text="""Can you process this invoice and extract data""")

In [7]:
response_mime_type = "application/json"
response_schema = {"type":"OBJECT","properties":{"CustomerName":{"type":"STRING"},"BillingAddress":{"type":"STRING"},"ShippingAddress":{"type":"STRING"},"CustomerAccount_ReferenceNumber":{"type":"STRING"},"VendorName":{"type":"STRING"},"VendorAddress":{"type":"STRING"},"VendorContactInformation":{"type":"STRING"},"VendorVAT_TaxRegistrationNumber":{"type":"STRING"},"TerminationNotificationPeriod":{"type":"STRING"},"InvoiceDate":{"type":"STRING","format":"date"},"PaymentDueDate":{"type":"STRING","format":"date"},"PurchaseOrderNumber":{"type":"STRING"},"OrderDate":{"type":"STRING","format":"date"},"SubTotal_NetAmount":{"type":"OBJECT","properties":{"Value":{"type":"NUMBER"},"CurrencyCode":{"type":"STRING"}}},"TAX_VATAmmount":{"type":"OBJECT","properties":{"Value":{"type":"NUMBER"},"CurrencyCode":{"type":"STRING"}}},"TotalAmmountDue":{"type":"OBJECT","properties":{"Value":{"type":"NUMBER"},"CurrencyCode":{"type":"STRING"}}},"TAX_VATRate":{"type":"NUMBER"},"Discounts_Promotions":{"type":"NUMBER"},"Shipping_DeliveryCharges":{"type":"OBJECT","properties":{"Value":{"type":"NUMBER"},"CurrencyCode":{"type":"STRING"}}},"LineItemDetails":{"type":"ARRAY","items":{"type":"OBJECT","properties":{"Description":{"type":"STRING"},"Quantity":{"type":"NUMBER"},"UnitPrice":{"type":"OBJECT","properties":{"Value":{"type":"NUMBER"},"CurrencyCode":{"type":"STRING"}}},"LineItemTotal":{"type":"OBJECT","properties":{"Value":{"type":"NUMBER"},"CurrencyCode":{"type":"STRING"}}},"Product_SKU Code":{"type":"STRING"}}}}},"required":["CustomerName","BillingAddress","ShippingAddress","CustomerAccount_ReferenceNumber","VendorName","VendorAddress","VendorContactInformation","VendorVAT_TaxRegistrationNumber","TerminationNotificationPeriod","PaymentDueDate","PurchaseOrderNumber","OrderDate","SubTotal_NetAmount","TAX_VATAmmount","TotalAmmountDue","TAX_VATRate","Discounts_Promotions","Shipping_DeliveryCharges","LineItemDetails"]}

In [8]:
generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    response_mime_type = response_mime_type,
    response_schema = response_schema,
    system_instruction=[types.Part.from_text(text=system_instruction)],
  )

#**Get files from GCS**

In [9]:
def list_file_uris_in_bucket(bucket_name):
  """Lists all file URIs in the specified Google Cloud Storage bucket.

  Args:
    bucket_name: The name of the GCS bucket.

  Returns:
    A list of file URIs (strings), or None if an error occurs.
  """
  try:
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blobs = bucket.list_blobs()

    file_uris = [f"gs://{bucket_name}/{blob.name}" for blob in blobs]
    return file_uris

  except Exception as e:
    print(f"An error occurred: {e}")
    return None

# **Use Gemini analyse files from gcs**

In [10]:
@retry(wait=wait_random_exponential(multiplier=1, max=60))
def classify_invoice(invoice_uri, prompt,generate_content_config):
  client = genai.Client(
      vertexai=True,
      project=project_id,
      location=region,
  )
  msg1_document1 = types.Part.from_uri(
      file_uri=invoice_uri,
      mime_type=mime_type,
  )
  contents = [
    types.Content(
      role="user",
      parts=[
        msg1_document1,
        prompt
      ]
    ),
  ]
  response = client.models.generate_content(
    model = model,
    contents = contents,
    config = generate_content_config)
  # print(response.text)
  return response.text

#**Run the code**

In [11]:
from os import times
client = bigquery.Client()
files = list_file_uris_in_bucket(prospectus_bucket)
print(len(files))
data = []
i=0
for pfile in files:
  i+=1
  print(pfile)
  output = classify_invoice(pfile, prompt, generate_content_config)
  timestamp = datetime.now().isoformat()
  data.append({"filepath": pfile, "details": output, "timestamp": timestamp})
  # print(output)
  if i%10 == 0:
    errors = client.insert_rows_json(
    f"{project_id}.{DATASET_ID}.{INVOICE_TABLE_ID}", data)
    print(errors)
    data = []
if len(data) > 0:
  errors = client.insert_rows_json(
      f"{project_id}.{DATASET_ID}.{INVOICE_TABLE_ID}", data)
  print(errors)
# print(output)

52
gs://bh-invoice-raw-sg/09072830.pdf
gs://bh-invoice-raw-sg/09075907.pdf
gs://bh-invoice-raw-sg/1530922.PDF
gs://bh-invoice-raw-sg/2025_06_24_09_30_13.pdf
gs://bh-invoice-raw-sg/211648.pdf
gs://bh-invoice-raw-sg/38497338.pdf
gs://bh-invoice-raw-sg/42007447 June invoice.pdf
gs://bh-invoice-raw-sg/AmazonBusiness_Invoice_1GMK-QFQN-VDWC.pdf
gs://bh-invoice-raw-sg/BREVAN HOWARD CAPITAL MANAGEMENT LP (UK) - (CXT) Invoice U3-001259.pdf
gs://bh-invoice-raw-sg/Bill61311_D72337872_X300487.pdf
[]
gs://bh-invoice-raw-sg/Brevan Howard Asset Management Services Limited inv 031083.pdf
gs://bh-invoice-raw-sg/Brevan Howard Capital Management Limited_48727_Invoice_Brevan Howard Capital Management Limited_260186807.pdf
gs://bh-invoice-raw-sg/Brevan Howard Capital Management Limited_48727_Reports_Brevan Howard Capital Management Limited_260187235.pdf
gs://bh-invoice-raw-sg/Brevard Howard Private Limited_50980565_9590033780_25062025_0517.pdf
gs://bh-invoice-raw-sg/CGEVC1250037_Facture 3101743_Valid JDR.p

# BH Address identification

In [12]:
import pandas as pd
import json
import ast

In [13]:
address_input = pd.read_csv("gs://bh-invoice-staticdata-sg/BH Invoice Coding - BH Companies.csv")
address_input.head()
address_input_md = address_input.to_markdown()

In [14]:
system_instruction = """You are an expert invoice processing agent at a hedgefund/Asset Manager. Your primary function is to accurately identify the Brevan Howard company from the invoice address.
You will use the address of the Brevan Howard compay the invoice is addressed to and use the data set below to identiy the company the invoice is addressed to.
Dont Hallucinate, in the repsonse from the table below get the company name and the refrence id.
The list of Brevan Howard Companies with their reference IDs and addresses is below:
<BH_company_address_data>: {address_input_md}

The outout shoud be JSON""".format(address_input_md=address_input_md)

In [15]:
prompt = types.Part.from_text(text="""Can you process this invoice and identify the address""")

In [16]:
response_mime_type = "application/json"
response_schema = {
    "type": "OBJECT",
    "properties": {
        "CompanyName": {
            "type": "STRING"
        },
        "CompanyAddress": {
            "type": "STRING"
        },
        "ReferenceID": {
            "type": "STRING"
        }
    },
    "required": [
        "CompanyName",
        "CompanyAddress",
        "ReferenceID"
    ]
}

In [17]:
generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    response_mime_type = response_mime_type,
    response_schema = response_schema,
    system_instruction=[types.Part.from_text(text=system_instruction)],
  )

In [23]:
from os import times
client = bigquery.Client()
files = list_file_uris_in_bucket(prospectus_bucket)
print(len(files))
data = []
i=0
for pfile in files:
  i+=1
  print(pfile)
  output = classify_invoice(pfile, prompt, generate_content_config)
  timestamp = datetime.now().isoformat()
  data.append({"filepath": pfile, "detail": output, "timestamp": timestamp})
  # print(output)
  if i%10 == 0:
    errors = client.insert_rows_json(
    f"{project_id}.{DATASET_ID}.{ADDRESS_TABLE_ID}", data)
    print(errors)
    data = []
if len(data) > 0:
  errors = client.insert_rows_json(
      f"{project_id}.{DATASET_ID}.{ADDRESS_TABLE_ID}", data)
  print(errors)
# print(output)

52
gs://bh-invoice-raw-sg/09072830.pdf
gs://bh-invoice-raw-sg/09075907.pdf
gs://bh-invoice-raw-sg/1530922.PDF
gs://bh-invoice-raw-sg/2025_06_24_09_30_13.pdf
gs://bh-invoice-raw-sg/211648.pdf
gs://bh-invoice-raw-sg/38497338.pdf
gs://bh-invoice-raw-sg/42007447 June invoice.pdf
gs://bh-invoice-raw-sg/AmazonBusiness_Invoice_1GMK-QFQN-VDWC.pdf
gs://bh-invoice-raw-sg/BREVAN HOWARD CAPITAL MANAGEMENT LP (UK) - (CXT) Invoice U3-001259.pdf
gs://bh-invoice-raw-sg/Bill61311_D72337872_X300487.pdf
[]
gs://bh-invoice-raw-sg/Brevan Howard Asset Management Services Limited inv 031083.pdf
gs://bh-invoice-raw-sg/Brevan Howard Capital Management Limited_48727_Invoice_Brevan Howard Capital Management Limited_260186807.pdf
gs://bh-invoice-raw-sg/Brevan Howard Capital Management Limited_48727_Reports_Brevan Howard Capital Management Limited_260187235.pdf
gs://bh-invoice-raw-sg/Brevard Howard Private Limited_50980565_9590033780_25062025_0517.pdf
gs://bh-invoice-raw-sg/CGEVC1250037_Facture 3101743_Valid JDR.p

# Supplier Coding

In [11]:
# supplier_coding_input = pd.read_csv("gs://bh-invoice-staticdata-sg/BH Invoice Coding - Supplier Coding Database.csv")
# supplier_coding_input.head()
# supplier_coding_input_md = supplier_coding_input.to_markdown()

In [12]:
system_instruction_orig = """You are an expert invoice processing agent at a hedgefund/Asset Manager. Your primary function is to accurately information from an invoice by matching it to the suplier coding database (listed below).
Based on this matching you will apply the Cost Center, Spend Category and Project ID to the output. Use the info in the invoice supplied with the prompt to identify the supplier from the supplier coding database give below.
Dont Hallucinate, in the repsonse from the table below get the Cost Center, Spend Category and Project ID. If a Project ID is not available set it to NA
The list of Brevan Howard Companies with their reference IDs and addresses is below:
<supplier coding database>: {supplier_coding_input_md}

The outout shoud be JSON"""


In [13]:
prompt = types.Part.from_text(text="""Can you process this invoice and identify the Cost Center, Spend Category and Project ID""")

In [14]:
response_mime_type = "application/json"
response_schema = {
    "type": "OBJECT",
    "properties": {
        "CostCenter": {
            "type": "STRING"
        },
        "SpendCategory": {
            "type": "STRING"
        },
        "ProjectID": {
            "type": "STRING"
        }
    },
    "required": [
        "CostCenter",
        "SpendCategory",
        "ProjectID"
    ]
}

In [15]:
generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    response_mime_type = response_mime_type,
    response_schema = response_schema,
    system_instruction=[types.Part.from_text(text=system_instruction)],
  )

In [ ]:
from os import times
client = bigquery.Client()
files = list_file_uris_in_bucket(prospectus_bucket)
print(len(files))
data = []
i=0
for pfile in files:
  i+=1
  print(pfile)

  sql = """SELECT filepath, detail['ReferenceID'], b.* FROM `cloud-llm-preview2.bh_invoice_sg.invoiceaddress` a
    inner join  `cloud-llm-preview2.bh_invoice_sg.suppliercodingdb` b
    on STRING(a.detail['ReferenceID']) = b.`Company ID`
    where filepath = "{pfile}"
    """.format(pfile=pfile)

  supplierdf = pandas_gbq.read_gbq(sql, project_id=project_id)
  system_instruction = system_instruction_orig.format(supplier_coding_input_md=supplierdf.to_markdown())
  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    response_mime_type = response_mime_type,
    response_schema = response_schema,
    system_instruction=[types.Part.from_text(text=system_instruction)],
  )

  output = classify_invoice(pfile, prompt, generate_content_config)
  timestamp = datetime.now().isoformat()
  data.append({"filepath": pfile, "detail": output, "timestamp": timestamp})
  # print(output)
  if i%10 == 0:
    errors = client.insert_rows_json(
    f"{project_id}.{DATASET_ID}.{SUPPLIERCODING_TABLE_ID}", data)
    print(errors)
    data = []
if len(data) > 0:
  errors = client.insert_rows_json(
      f"{project_id}.{DATASET_ID}.{SUPPLIERCODING_TABLE_ID}", data)
  print(errors)
# print(output)